### Test Description:
This Python test script is designed to check the response from an Arduino device connected through UART communication. This basic communications test for message response, checksum and multimessage sending.

### GIT issue
https://github.com/KKallas/ApolloFirmware/issues/7

# How to Use the Test Script:

## Prerequisites:
Use Jupyter-Lab and Serial library
Make sure the lamp is connected to computer Serial port (USB)

# Run the Test:
Execute the Python cells below:
The script will send the multipart test message ```DS00->//DS01->Hello:101//000:::``` to the Arduino device.
It will receive and analyze the responses from the Arduino.

* Check if serial port is connected
* Check if lamp is powered
* Check if arduino ide or something else is not hogging the com port

# Setup

In [43]:
def calculate_checksum(input_string):
    checksum = 0
    
    for char in input_string:
        checksum += ord(char)
        checksum &= 0xFFFF  # Ensure it remains a 2-byte value
        
    return checksum

command = "DS10->Hello World"
checksum = calculate_checksum(command + '//')
print(command + '//' + '{:04X}'.format(checksum) + ':::')

DS10->Hello World//05DD:::


In [44]:
import serial
import time

# Define the UART settings
baud_rate = 115200 #250000  # Baud rate (bits per second)
serial_port = 'COM6'  # Replace with your Arduino's serial port
# Open the serial port
ser = serial.Serial(serial_port, baud_rate, timeout=2, dsrdtr=None)
# Create a function to send a multi-command message and receive responses
def communicate_with_arduino(commands):
    try:
        # Clear any initial data in the buffer
        while ser.in_waiting > 0:
            ser.readline()
            
        ser.write(multi_command_message.encode())
        responses = []
        
        # Read and store responses until a timeout occurs
        while True:
            response = ser.readline().decode()
            if not response:
                break
            responses.append(response)

        return responses

    except Exception as e:
        print(e)
        return ["ERROR"]

# Single Package tests

## Response test
This test sends a multi-command message to an Arduino with checksum replaced with 0000, searching for a response that starts with "DS10" and boasts a payload commencing with "Hello World," affirming the test's success if such a response materializes, or pronouncing failure if no such package emerges from the Arduino's responses.

In [45]:
# Define the multi-command message with multiple commands
multi_command_message = "DS99->Hello World//0000:::\n"

# Send the multi-command message to Arduino and receive responses
responses = communicate_with_arduino(multi_command_message)

# Check if any response starts with "DS10" and has a payload starting with "Apollo"
found_matching_response = False
for response in responses:
    if response.startswith("DS10") and response.split("->")[1].startswith("Hello World"):
        found_matching_response = True
        break

if found_matching_response:
    print("\033[32mTest passed.\033[0m Arduino responded with a matching package.")
else:
    print("Test failed.\033[0m No matching package found in Arduino's responses.")


Test passed. Arduino responded with a matching package.


## Checksum test
This test extracts a hexadecimal checksum from the first response, converts it to an integer, and then compares it to a recalculated checksum from the rest of the response. If they match, the test triumphs with a declaration of the Arduino's correctness; otherwise, it confesses failure due to a checksum mismatch.

In [56]:
originalChecksumAsInt = int(responses[0][-9:-5],16)
if calculate_checksum(responses[0][:-9]) == originalChecksumAsInt:
    print("\033[32mTest passed.\033[0m Arduino responded with a correct checksum.")
else:
    print("\033[31mTest failed.\033[0m Checksum does not match.")

Test passed. Arduino responded with a correct checksum.


## Checksummed response
This test sends a multi-command message to an Arduino with **correct checkum** searching for a response that starts with "DS10" and boasts a payload commencing with "Hello World," affirming the test's success if such a response materializes, or pronouncing failure if no such package emerges from the Arduino's responses.

In [57]:
# Define the multi-command message with multiple commands
multi_command_message = "DS99->Hello World//05EE:::\n"

# Send the multi-command message to Arduino and receive responses
responses = communicate_with_arduino(multi_command_message)

# Check if any response starts with "DS10" and has a payload starting with "Apollo"
found_matching_response = False
for response in responses:
    if response.startswith("DS10") and response.split("->")[1].startswith("Hello World"):
        found_matching_response = True
        break

if found_matching_response:
    print("\033[32mTest passed.\033[0m Arduino responded with a matching package.")
else:
    print("\033[31mTest failed.\033[0m No matching package found in Arduino's responses.")


Test passed. Arduino responded with a matching package.


## Incorrect checksummed response
This test sends a multi-command message to an Arduino with **incorrect checkum** searching for a response that starts with "DS10" and boasts a payload commencing with "Hello World," affirming the test's success if such a response materializes, or pronouncing failure if no such package emerges from the Arduino's responses.

In [58]:
# Define the multi-command message with multiple commands
multi_command_message = "DS99->Hello World//05A1:::\n"

# Send the multi-command message to Arduino and receive responses
responses = communicate_with_arduino(multi_command_message)

# Check if any response starts with "DS10" and has a payload starting with "Apollo"
found_matching_response = False
for response in responses:
    if response.startswith("DS10") and response.split("->")[1].startswith("Hello World"):
        found_matching_response = True
        break

if found_matching_response:
    print("\033[31mTest failed.\033[0m Arduino responded with a matching package.")
else:
    print("\033[32mTest passed.\033[0m Arduino did not respond with a matching package.")


Test passed. Arduino did not respond with a matching package.


# Multipackage tests

## 2 Payload test
This test dispatches a multi-command message to the Arduino, eagerly awaiting a response. It meticulously examines the second response, scrutinizing it for the presence of the cherished phrase "This is me." If this revered phrase emerges from the Arduino's reply, the test graciously bestows a verdant success

In [83]:
# Define the multi-command message with multiple commands
multi_command_message = "DS99->Hello World//DS99->This is me//0B46:::\n"

# Send the multi-command message to Arduino and receive responses
responses = communicate_with_arduino(multi_command_message)

if responses[1][6:16] == "This is me":
    print("\033[32mTest passed.\033[0m Arduino responded with a matching package.")
else:
    print("\033[31mTest failed.\033[0m No matching package found in Arduino's responses.")

Test passed. Arduino responded with a matching package.


No Parameters in the 1st command

In [84]:
# Define the multi-command message with multiple commands
multi_command_message = "DS99->//DS99->This is me//072A:::\n"

# Send the multi-command message to Arduino and receive responses
responses = communicate_with_arduino(multi_command_message)

if responses[1][6:16] == "This is me":
    print("\033[32mTest passed.\033[0m Arduino responded with a matching package.")
else:
    print("\033[31mTest failed.\033[0m No matching package found in Arduino's responses.")

Test passed. Arduino responded with a matching package.


No Parameters and no paramter seprator in the 1st command

In [85]:
# Define the multi-command message with multiple commands
multi_command_message = "DS99//DS99->This is me//06BF:::\n"

# Send the multi-command message to Arduino and receive responses
responses = communicate_with_arduino(multi_command_message)

if responses[1][6:16] == "This is me":
    print("\033[32mTest passed.\033[0m Arduino responded with a matching package.")
else:
    print("\033[31mTest failed.\033[0m No matching package found in Arduino's responses.")

Test passed. Arduino responded with a matching package.


In [42]:
ser.close()